#### Exercise 1
Familiarize with the [`Tree`](https://www.nltk.org/api/nltk.tree.tree.html) class.

- Consult the documentation for more detail (and other methods)
- Try each of the "core" methods listed above
    - see production rules, leaves, and pos

######   "Core" Methods

- `fromstring()` reads a bracketed tree string and return the resulting tree

- `productions()` generate the productions that correspond to the non-terminal nodes of the tree.
    - For each subtree of the form `(P: C1 C2 ... Cn)` this produces a production of the form `P -> C1 C2 ... Cn`.

- `label()` returns the node label of the tree

- `subtrees()` generates all the subtrees of this tree

- `pos()` return a sequence of pos-tagged words extracted from the tree.

- `leaves()` returns the leaves of the tree.

- `flatten()` return a flat version of the tree, with all non-root non-terminals removed.

In [8]:
# Productions
print('Productions:' )
print(tree.productions())
# Label
print('Label:' )
print(tree[1].label())
# Pos tags
print('Pos Tags:' )
print(tree.pos())
# Leaves 
print('Leaves:' )
print(tree.leaves())
# Flatten
print('Flatten:' )
print(tree.flatten())

Productions:
[S -> NP VP, NP -> PRON, PRON -> 'I', VP -> V NP PP, V -> 'saw', NP -> Det N, Det -> 'the', N -> 'man', PP -> P NP, P -> 'with', NP -> Det N, Det -> 'a', N -> 'telescope']
Label:
VP
Pos Tags:
[('I', 'PRON'), ('saw', 'V'), ('the', 'Det'), ('man', 'N'), ('with', 'P'), ('a', 'Det'), ('telescope', 'N')]
Leaves:
['I', 'saw', 'the', 'man', 'with', 'a', 'telescope']
Flatten:
(S I saw the man with a telescope)


#### Exercise 2

- Define grammar that covers the following sentences.

    - show flights from new york to los angeles
    - list flights from new york to los angeles
    - show flights from new york
    - list flights to los angeles
    - list flights
- Use one of the parsers to parse the sentences (i.e. test your grammar)

**Note:** <br>
- start from Verb Pharse
- Prepositional Phrase (PP) are composed of a preposition and a Noun Phrase 
- 'los' and 'angeles' are two consecutive nouns (N) (same for new york)




In [18]:
# test setenteces
test_sents = [
    "show flights from new york to los angeles", 
    "list flights from new york to los angeles",
    "show flights from new york",
    "list flights to los angeles",
    "list flights"]

In [19]:
rules = [
    'S -> VP',
    'VP -> V NP | V NP PP | V NP PP PP',
    'NP -> N | N N' ,
    'PP -> P NP | PP',
    'P -> "from" | "to"',
    'N -> "york" | "angeles" | "flights" | "los" | "new" ' ,
    'V -> "show" | "list"' 
]

toy_grammar = nltk.CFG.fromstring(rules)

In [20]:
parser = nltk.ChartParser(toy_grammar)

for sent in test_sents:
    for tree in parser.parse(sent.split()):
        print(tree.pretty_print())

                   S                              
                   |                               
                   VP                             
  _________________|________________               
 |      |          PP               PP            
 |      |      ____|___          ___|___           
 |      NP    |        NP       |       NP        
 |      |     |     ___|___     |    ___|_____     
 V      N     P    N       N    P   N         N   
 |      |     |    |       |    |   |         |    
show flights from new     york  to los     angeles

None
                   S                              
                   |                               
                   VP                             
  _________________|________________               
 |      |          PP               |             
 |      |          |                |              
 |      |          PP               PP            
 |      |      ____|___          ___|___           
 |      NP    | 

#### Exercise 3
Parse the sentences from the excercise above using **ATIS_GRAMMAR**
- try different parsers
- output the number of parses each sentence yields 

In [27]:
atis_parser = nltk.ChartParser(atis_grammar)
for sent in test_sents:
    parses = atis_parser.parse(sent.split())
    print(len(list(parses)))

3
3
1
1
1


### Exercise 4

- Try different parser to parse the sentences from the exercises above
- Compare assigned probabilities
- Compare time it takes to parse sentences

In [35]:
tokenized_sentence = "show me flights from New York to Los Angeles".split()

In [36]:
%%time
parser = nltk.ViterbiParser(grammar)
for tree in parser.parse(tokenized_sentence):
    print(tree)

(S
  (NP-SBJ (NN show))
  (NP-PRD
    (NP (NP (PRP me)) (NNS flights))
    (PP
      (IN from)
      (NP
        (NP (NNP New) (NNP York))
        (PP (TO to) (NP (NNP Los) (NNP Angeles))))))) (p=2.0478e-35)
CPU times: user 6.2 s, sys: 2.67 ms, total: 6.2 s
Wall time: 6.22 s


In [42]:
%%time
parser = nltk.InsideChartParser(grammar, beam_size=10000)
for tree in parser.parse(tokenized_sentence):
    print(tree)

(S
  (PRN (SINV (VP (VB show)) (NP-SBJ (PRP me))))
  (NP-SBJ-2 (NNS flights))
  (ADVP-TMP (IN from))
  (VP
    (JJ New)
    (NP (NNP York))
    (PP (IN to) (NP-LGS (NNP Los) (NNP Angeles))))) (p=1.79623e-46)
(S
  (PRN (SINV (VP (VBP show)) (NP-SBJ (PRP me))))
  (NP-SBJ-2 (NNS flights))
  (ADVP-TMP (IN from))
  (VP
    (JJ New)
    (NP (NNP York))
    (PP (IN to) (NP-LGS (NNP Los) (NNP Angeles))))) (p=1.56276e-46)
(S
  (PRN (SINV (VP (NN show)) (NP-SBJ (PRP me))))
  (NP-SBJ-2 (NNS flights))
  (ADVP-TMP (IN from))
  (VP
    (JJ New)
    (NP (NNP York))
    (PP (IN to) (NP-LGS (NNP Los) (NNP Angeles))))) (p=4.57328e-48)
(S
  (PRN (SINV (VP (VP (VB show))) (NP-SBJ (PRP me))))
  (NP-SBJ-2 (NNS flights))
  (ADVP-TMP (IN from))
  (VP
    (JJ New)
    (NP (NNP York))
    (PP (IN to) (NP-LGS (NNP Los) (NNP Angeles))))) (p=8.66548e-50)
(S
  (PRN (SINV (VP (VP (VBP show))) (NP-SBJ (PRP me))))
  (NP-SBJ-2 (NNS flights))
  (ADVP-TMP (IN from))
  (VP
    (JJ New)
    (NP (NNP York))
    (PP (IN to) 

In [45]:
%%time
parser = nltk.RandomChartParser(grammar, beam_size=100000)
for tree in parser.parse(tokenized_sentence):
    print(tree)

CPU times: user 21min 47s, sys: 983 ms, total: 21min 48s
Wall time: 21min 50s
